In [15]:
import severability
import networkx as nx
import numpy as np

In [16]:
G = nx.read_edgelist("congress_network/congress.edgelist", create_using=nx.DiGraph, nodetype=int)

In [17]:
# get adjacency matrix and convert to np.matrix
A = nx.to_numpy_array(G)
A = np.matrix(A)

# compute transition matrix
P = severability.transition_matrix(A)

In [18]:
import json

with open("congress_network/congress_network_data.json", "r") as f:
    data = json.load(f)
    usernames = data[0]['usernameList']
    f.close()

assert len(usernames) == A.shape[0], "Number of usernames does not match the number of nodes in the graph."

In [19]:
class Congressperson:
    def __init__(self, username, index, max_t = 5):
        self.username = username
        self.index = index
        self.max_t = max_t
        self.time_series = dict.fromkeys(range(1, max_t + 1), None)
    
    def __repr__(self):
        return f"Congressperson(username={self.username}, index={self.index})"
    
    def calculate_components(self, P):
        print(f"Calculating components for {self.username} at index {self.index}")
        for t in range(1, self.max_t + 1):
            (c, sev) = severability.node_component(P, self.index, t, max_size=10)
            print(f"t={t}, component={[usernames[i] for i in c]}, severability={sev}")
            self.time_series[t] = (c, sev)

    def mean_severability(self):
        return np.mean([sev for _, sev in self.time_series.values() if sev is not None])

In [20]:
congresspeople = [ Congressperson(username, index) for index, username in enumerate(usernames) ]

In [21]:
for p in congresspeople:
    p.calculate_components(P)

Calculating components for SenatorBaldwin at index 0
t=1, component=['SenatorBaldwin', 'CoryBooker', 'SenCapito', 'senrobportman'], severability=0.4480514632744105
t=2, component=['SenatorBaldwin', 'SenCortezMasto', 'SenTomCotton', 'SenBillCassidy', 'SenSherrodBrown', 'SenCapito', 'SenJohnBarrasso', 'SenatorCardin', 'SenBobCasey', 'RepJamesComer'], severability=0.4589288084082493
t=3, component=['SenatorBaldwin', 'SenCortezMasto', 'SenTomCotton', 'SenBillCassidy', 'SenSherrodBrown', 'SenBobCasey', 'SenJohnBarrasso', 'SenatorCardin', 'SenCapito', 'SenatorCantwell'], severability=0.4713241388554552
t=4, component=['SenatorBaldwin', 'SenCortezMasto', 'SenTomCotton', 'SenBillCassidy', 'SenatorCarper', 'SenatorCardin', 'SenSherrodBrown', 'SenBobCasey', 'SenJohnBarrasso', 'SenAngusKing'], severability=0.48436258374179125
t=5, component=['SenatorBaldwin', 'SenCortezMasto', 'SenTomCotton', 'SenBillCassidy', 'SenatorCarper', 'SenAlexPadilla', 'SenSherrodBrown', 'SenBobCasey', 'SenatorCantwell',

In [23]:
congresspeople = sorted(congresspeople, key=lambda p: p.mean_severability())
print("Congresspeople sorted by mean severability:")
for p in congresspeople:
    print(f"    {p.username}: {p.mean_severability()}")

Congresspeople sorted by mean severability:
    RepJohnYarmuth: 0.1783527450756838
    RepHankJohnson: 0.18170147699374928
    RepValDemings: 0.18396161031979835
    RepWilson: 0.3167517423544043
    RobWittman: 0.3565539575324429
    RepSusanWild: 0.36138043815437665
    RepTimmons: 0.36968160385741416
    RepLaTurner: 0.37828185932859976
    RepSwalwell: 0.379418081634623
    RepDWStweets: 0.3826711036377384
    rep_stevewomack: 0.3850987244262756
    RepSires: 0.4303149840281629
    RepPressley: 0.43723386804830355
    JimLangevin: 0.44117161022849655
    RepChipRoy: 0.44117161022849655
    RepBryanSteil: 0.44367118274224016
    RepWesterman: 0.44520895173520414
    maziehirono: 0.44700362502850755
    SenatorRomney: 0.4471968595981982
    RepEscobar: 0.44757352223083435
    RepStefanik: 0.44780253676778087
    RepGosar: 0.44927761427276636
    RepRosendale: 0.4494656237190906
    MikeKellyPA: 0.44957786965572055
    RepJoeMorelle: 0.450034336221961
    JohnCornyn: 0.451065937134703